# Installating R on WinPython

#### This procedure applys for Winpython  (Version of December 2015 and after) 
### 1 - Downloading and Installing R binary in the right place

In [1]:
import os
import sys
import io

In [2]:
os.environ["tmp_Rdirectory"]= "R" 
os.environ["R_HOME"] = os.environ["WINPYDIR"]+ "\\..\\tools\\R\\" 
os.environ["R_HOMEbin"]=os.environ["R_HOME"] + "bin" 
# for installation we need this
os.environ["tmp_Rbase"]=os.path.join(os.path.split(os.environ["WINPYDIR"])[0]  , 'tools','R' )  

In [3]:
# downloading R-3.2.3 (65 380 480 Bytes, may takes a few minutes)
try:
    import urllib.request as urllib2  # Python 3
except:
    import urllib2  # Python 2
if 'amd64' in sys.version.lower():
    r_comp ='/COMPONENTS="main,x64,translations'
else:
    r_comp ='/COMPONENTS="main,i386,translations'

# specify R binary and hash
r_url = "https://cran.r-project.org/bin/windows/base/R-3.2.3-win.exe"
hashes=("5572ce407831ce9156485d4f896ad616", "b86077268e4aa5498482f4514202009009ddf947" )

# specify target location
r_binary = os.path.basename(r_url)
r_installer = os.environ["WINPYDIR"]+"\\..\\tools\\"+r_binary
os.environ["r_installer"] = r_installer

# Download
g = urllib2.urlopen(r_url) 
with io.open(r_installer, 'wb') as f:
    f.write(g.read())
g.close
g = None

In [4]:
#checking it's there
!dir %r_installer%

 Le volume dans le lecteur D s'appelle DATA
 Le num‚ro de s‚rie du volume est 10BD-2ADB

 R‚pertoire de D:\result_tests\winpython-3.5.1.amd64\tools

03/01/2016  12:19        65ÿ380ÿ480 R-3.2.3-win.exe
               1 fichier(s)       65ÿ380ÿ480 octets
               0 R‚p(s)  133ÿ833ÿ940ÿ992 octets libres


In [5]:
# checking it's the official R-3.2.3 
import hashlib
def give_hash(of_file, with_this):
    with io.open(r_installer, 'rb') as f:
        return with_this(f.read()).hexdigest()  
print (" "*12+"MD5"+" "*(32-12-3)+" "+" "*15+"SHA-1"+" "*(40-15-5)+"\n"+"-"*32+" "+"-"*40)

print ("%s %s %s" % (give_hash(r_installer, hashlib.md5) , give_hash(r_installer, hashlib.sha1),r_installer))
assert give_hash(r_installer, hashlib.md5) == hashes[0]
assert give_hash(r_installer, hashlib.sha1) == hashes[1]

            MD5                                 SHA-1                    
-------------------------------- ----------------------------------------
5572ce407831ce9156485d4f896ad616 b86077268e4aa5498482f4514202009009ddf947 D:\result_tests\winpython-3.5.1.amd64\python-3.5.1.amd64\..\tools\R-3.2.3-win.exe


In [6]:
# let's install it 
# If you are "USB life style", or multi-winpython
#   ==> CLICK the OPTION "Don't create a StartMenuFolder' <== (when it will show up)
os.environ["tmp_R_comp"]=r_comp
!start cmd /C %r_installer% /DIR=%tmp_Rbase% %tmp_R_comp%

## During Installation (if you wan't to move the R installation after)


Select  "Don't create a Start Menu Folder"

Un-select "Create a desktop icon"

Un-select "Save version number in registery"


<img src="https://raw.githubusercontent.com/stonebig/winpython_afterdoc/master/examples/images/r_setup_unclick_shortcut.GIF">

### 2 - create a R_launcher and install irkernel

In [7]:
import os
import sys
import io
# let's create a R launcher  
r_launcher = r"""
@echo off
call %~dp0env.bat
rscript %*
"""
r_launcher_bat = os.environ["WINPYDIR"]+"\\..\\scripts\\R_launcher.bat"

# let's create a R init script
# in manual command line, you can use repos = c('http://irkernel.github.io/', getOption('repos'))
r_initialization = r"""
install.packages(c('rzmq','repr','IRkernel','IRdisplay'), repos = c('http://irkernel.github.io/', repos='http://cran.rstudio.com/'))
library('rzmq')
library('repr')
library('IRkernel')
library('IRdisplay')
IRkernel::installspec()
"""
r_initialization_r = os.path.normpath(os.environ["WINPYDIR"]+"\\..\\scripts\\R_initialization.r")


for i in [(r_launcher,r_launcher_bat), (r_initialization, r_initialization_r)]:
    with io.open(i[1], 'w', encoding = sys.getdefaultencoding() ) as f:
        for line in i[0].splitlines():
            f.write('%s\n' %  line  )


In [8]:
# Launch Rkernel setup
os.environ["r_initialization_r"] = r_initialization_r
!start cmd /C %WINPYDIR%\\..\\scripts\\R_launcher.bat --no-restore --no-save  %r_initialization_r%  

In [9]:
# make RKernel a movable installation with the rest of WinPython 
from winpython import utils
base_winpython = os.path.dirname(os.path.normpath(os.environ["WINPYDIR"]))
rkernel_json=(base_winpython+"\\settings\\kernels\\ir\\kernel.json")

# so we get "argv": ["{prefix}/../tools/R/bin/x64/R"
utils.patch_sourcefile(rkernel_json, base_winpython.replace("\\","/"), r'{prefix}/..', silent_mode=False)

patching  D:\result_tests\winpython-3.5.1.amd64\settings\kernels\ir\kernel.json from D:/result_tests/winpython-3.5.1.amd64 to {prefix}/..


### 3- Install a R package via a IPython  Kernel

In [10]:
%load_ext rpy2.ipython

#vitals: 'dplyr', 'R.utils', 'nycflights13'
# installation takes 2 minutes
%R install.packages(c('dplyr','R.utils', 'nycflights13'), repos='http://cran.rstudio.com/') 

### 4- Small demo via R magic

In [11]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [12]:
%%R
library('dplyr')
library('nycflights13') 
write.csv(flights, "flights.csv")

In [13]:
%R head(flights) 

,year,month,day,dep_time,dep_delay,arr_time,arr_delay,carrier,tailnum,flight,origin,dest,air_time,distance,hour,minute
1,2013,1,1,517,2,830,11,UA,N14228,1545,EWR,IAH,227,1400,5,17
2,2013,1,1,533,4,850,20,UA,N24211,1714,LGA,IAH,227,1416,5,33
3,2013,1,1,542,2,923,33,AA,N619AA,1141,JFK,MIA,160,1089,5,42
4,2013,1,1,544,-1,1004,-18,B6,N804JB,725,JFK,BQN,183,1576,5,44
5,2013,1,1,554,-6,812,-25,DL,N668DN,461,LGA,ATL,116,762,5,54
6,2013,1,1,554,-4,740,12,UA,N39463,1696,EWR,ORD,150,719,5,54


In [14]:
%R airports %>% mutate(dest = faa) %>% semi_join(flights) %>% head

,faa,name,lat,lon,alt,tz,dst,dest
1,IAH,George Bush Intercontinental,29.984433,-95.341442,97,-6,A,IAH
2,MIA,Miami Intl,25.793250,-80.290556,8,-5,A,MIA
3,ATL,Hartsfield Jackson Atlanta Intl,33.636719,-84.428067,1026,-5,A,ATL
4,ORD,Chicago Ohare Intl,41.978603,-87.904842,668,-6,A,ORD
5,FLL,Fort Lauderdale Hollywood Intl,26.072583,-80.152750,9,-5,A,FLL
6,IAD,Washington Dulles Intl,38.944533,-77.455811,313,-5,A,IAD


### 5 - Installing the very best of R pakages (optional, you will start to get a really big directory)


In [15]:
# essentials: 'tidyr', 'shiny', 'ggplot2', 'caret' , 'nnet'  
# remaining of Hadley Wickahm "stack" (https://github.com/rstudio)
%R install.packages(c('tidyr', 'ggplot2',  'shiny','caret' , 'nnet'), repos='https://cran.rstudio.com/') 
%R install.packages(c('knitr', 'purrr', 'readr', 'readxl'), repos='https://cran.rstudio.com/')
%R install.packages(c('rvest', 'lubridate', 'ggvis'), repos='https://cran.rstudio.com/')

# TRAINING = online training book http://r4ds.had.co.nz/ (or https://github.com/hadley/r4ds)

### 6 - Relaunch Jupyter Notebook to get a R kernel option
launch a new notebook of "R" type, and type in it:
    
library('dplyr')

library('nycflights13') 

head('flights')

### 9 - To Un-install  / Re-install R (or other trouble-shooting)

- launch  winpython**\tools\R\unins000.exe

- delete the directory winpython**\tools\R

- re-install
